In [ ]:
from functions import *
from imports import *
from constants import *

In [ ]:
# Initialisation du logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

# Launching Automation Driver and Website

In [ ]:
driver = create_driver()
main_page = driver.current_url
wait5 = WebDriverWait(driver=driver, timeout=5)
wait10 = WebDriverWait(driver=driver, timeout=10)
# Créer une instance d'ActionChains pour interagir avec l'élément
actions = ActionChains(driver)

# Connection to Account

In [ ]:
button_mi_cuenta = driver.find_element(By.XPATH, 
                                       "//a[normalize-space()='Mi cuenta']")
if button_mi_cuenta.is_enabled():
    button_mi_cuenta.click()
    time_sleep()


email_zone = driver.find_element(By.XPATH, 
        "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@id='username']")
email_zone.send_keys(my_email)
time_sleep()
password_zone = driver.find_element(By.XPATH, 
    "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@id='password']")
password_zone.send_keys(my_password)
time_sleep()
recordame = driver.find_element(By.XPATH, 
        "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//input[@name='rememberme']")
if recordame.is_enabled() and not recordame.is_selected():
    recordame.click()
time_sleep()
ingresar = driver.find_element(By.XPATH,
    "//form[@action='https://renzo.com.co/mi-cuenta/?action=login']//button[@name='login'][normalize-space()='Ingresar']")
if ingresar.is_enabled() and not ingresar.is_selected():
    ingresar.click()
    WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    time_sleep()    
    driver.get(main_page)

# Query Statement

In [ ]:
SEARCH_QUERY = "camisa lino hombre"

# Query Search

In [ ]:
try:
    print("🌍 Navigating to main page...")
    driver.get(main_page)
    time_sleep()
except Exception as e:
    print(f"❌ Failed to load main page: {e}")
    raise

try:
    print("🔁 Translating query...")
    print(f"🔤 Translated query: {SEARCH_QUERY}")
except Exception as e:
    print(f"❌ Error translating query: {e}")
    raise

try:
    print("🔍 Looking for the search button...")
    
    # Attendre que la div contenant le bouton de recherche soit visible
    search_button_div = wait5.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.search-button.basel-search-full-screen")))

    # Trouver l'élément <a> à l'intérieur de cette div avec le bon aria-label
    search_link = search_button_div.find_element(By.XPATH, ".//a[@aria-label='Search']")
    
    # Attendre que l'élément soit cliquable
    wait5.until(EC.element_to_be_clickable(search_link))

    # Cliquer directement sur l'élément <a>
    search_link.click()  # Utiliser la méthode click directement
    print("✅ Search button clicked.")
except Exception as e:
    print(f"❌ Failed to click the search button: {e}")

try:
    print("🔍 Locating search input field by ID 's'...")
    search_field = wait10.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#s.search-field.basel-search-inited")))

    print("📝 Sending keys to search input...")
    search_field.clear()    
    time_sleep()
    print("📝 Typing and submitting search query...")
    search_field.send_keys(SEARCH_QUERY)
    search_field.send_keys(Keys.ENTER)
except Exception as e:
    print(f"❌ Failed to input and submit search query: {e}")
    raise


In [ ]:
try:
    print("🌍 Navigating to main page...")
    driver.get(main_page)
    time_sleep()
except Exception as e:
    print(f"❌ Failed to load main page: {e}")
    raise


try:
    print("🔍 Looking for the search button...")
    search_button = wait5.until(EC.presence_of_element_located(
    (By.XPATH, "//button[contains(text(), 'Search')]")))

    driver.execute_script("arguments[0].click();", search_button)
    print("✅ Search button clicked with JS.")
except Exception as e:
    print(f"❌ Failed to click the search button: {e}")

try:
    print("🔍 Locating search input field by ID 's'...")
    search_field = wait10.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#s.search-field.basel-search-inited")))

    print("📝 Sending keys to search input...")
    search_field.clear()    
    time_sleep()
    print("📝 Typing and submitting search query...")
    search_field.send_keys(query)
    search_field.send_keys(Keys.ENTER)
except Exception as e:
    print(f"❌ Failed to input and submit search query: {e}")
    raise

# Data Collection

In [ ]:
dict_results = {}
count = 0

######## COLLECT ALL CARDS #############################################################################################################################################################

while True:

    try:
        cards = wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product")))
    except Exception as e:
        logger.error(f"❌ Failed to retrieve product cards: {e}")
        break

    for index in range(len(cards)):
        count += 1
        datas = {}
        datas["Promo"] = False  # Default value

        try:
            cards = wait5.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".product")))
            card = cards[index]
        except Exception as e:
            logger.error(f"❌ Error retrieving card {index}: {e}")
            break  # STOP ALL

######## TITLE #############################################################################################################################################################

        try:
            logger.info("📜 Retrieving product title...")
            title = card.find_element(By.XPATH, ".//h3[contains(@class, 'product-title')]").text
            datas["title"] = title
            logger.info(f"✅ Title retrieved: {title}")
        except Exception as e:
            logger.error(f"❌ Error retrieving title: {e}")
            break

######## COLORS #############################################################################################################################################################

        try:
            logger.info("🎨 Retrieving available colors...")
            zone_color = card.find_element(By.XPATH, ".//div[contains(@class, 'swatches-wrapper')]")
            colors_part = card.find_elements(By.XPATH, ".//div[contains(@class, 'swatch-on-grid') and contains(@class, 'swatch-has-image') and @data-image-src]")
            available_colors = [c.text.strip() for c in colors_part]
            datas["available colors"] = available_colors
            logger.info(f"✅ Available colors: {', '.join(available_colors)}")

            for color in colors_part:
                try:
                    color_name = color.text.strip()
                    driver.execute_script("arguments[0].scrollIntoView(true);", color)
                    driver.execute_script("arguments[0].click();", color)

                    image_url = wait5.until(EC.presence_of_element_located(
                        (By.XPATH, ".//img[contains(@class, 'attachment-woocommerce_thumbnail')]"))).get_attribute('src')
                    datas[f"Image_url_{color_name}"] = image_url
                    logger.info(f"✅ Image retrieved for color {color_name}")
                except Exception as e:
                    logger.error(f"❌ Error with color {color.text if color else 'UNKNOWN'}: {e}")
                    break  # STOP ALL

        except Exception as e:
            logger.error(f"❌ Error processing colors: {e}")
            break

######## PRICE #############################################################################################################################################################
        try:
            fresh_card = driver.find_elements(By.CSS_SELECTOR, ".product")[index]
            price_elements = fresh_card.find_elements(By.XPATH, ".//span[contains(@class, 'price')]//bdi")  # Find all price elements
            prices = [price.text.strip().replace('$', '').strip() for price in price_elements if price.text.strip()]
            
            if prices:
                # Créer un dictionnaire temporaire pour associer chaque prix string à sa version float
                price_dict = {price: float(price.replace(',', '.')) for price in prices}

                # Trouver le prix le plus bas parmi les floatifiés
                lowest_price_float = min(price_dict.values())

                # Conserver le prix string original
                lowest_price_str = next(price for price, value in price_dict.items() if value == lowest_price_float)

                # Ajouter le prix dans le dictionnaire des données (prix en string original)
                datas["price"] = lowest_price_str

                # Calculer la réduction en pourcentage, si deux prix existent
                if len(price_dict) > 1:
                    highest_price_float = max(price_dict.values())
                    discount = round((1 - (lowest_price_float / highest_price_float)) * 100)
                    datas["Promo"] = f"{discount}%"

                # Afficher dans les logs le prix en version string
                logger.info(f"✅ Price found: {lowest_price_str}")
            else:
                logger.error("❌ No valid price found.")
                break
        except Exception as e:
            logger.error(f"❌ Error retrieving price: {e}")
            break



######## GET TO PRODUCT PAGE #############################################################################################################################################################

        try:
            link_to_details = card.find_element(By.XPATH, ".//h3[contains(@class, 'product-title')]/a").get_attribute("href")
            logger.info(f"✅ Product detail link retrieved: {link_to_details}")
        except Exception as e:
            logger.error(f"❌ Error retrieving link: {e}")
            break

        try:
            driver.get(link_to_details)
            logger.info("🌐 Opening product detail page...")
        except Exception as e:
            logger.error(f"❌ Error opening product page: {e}")
            break

######## GET AVAILABLE SIZES #############################################################################################################################################################

        try:
            zone_sizes = wait5.until(EC.presence_of_element_located(
                (By.XPATH, "//ul[contains(@aria-label,'Talla')]")))
            available_sizes = zone_sizes.find_elements(By.XPATH, ".//li[not(contains(@class, 'disabled'))]")
            datas["available sizes"] = [size.get_attribute("title") for size in available_sizes if size.get_attribute("title")]
            logger.info(f"✅ Available sizes: {', '.join(datas['available sizes'])}")
        except Exception as e:
            logger.error(f"❌ Error retrieving sizes: {e}")
            break

        try:
            driver.back()
            wait5.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".product")))
            logger.info("🔙 Back to product list...")
        except Exception as e:
            logger.error(f"❌ Error returning to product list: {e}")
            break

        dict_results[count] = datas
        logger.info(f"✅ Product {count} added to results.")

######## NEXT PAGE #############################################################################################################################################################

    try:
        logger.info("🔍 Looking for 'Next' button...")
        next_page = wait5.until(EC.presence_of_element_located(
            (By.XPATH, "//a[contains(@class, 'next page-numbers')]")))
        driver.execute_script("arguments[0].click();", next_page)
        logger.info("➡️ Moving to the next page...")
    except NoSuchElementException:
        logger.info("❌ No 'Next' button found, stopping pagination.")
        break
    except Exception as e:
        logger.error(f"Error clicking on 'Next' button: {e}")
        break


# Normalize the Data Results Dictionary

In [ ]:
# Traiter le dictionnaire pour que les images soient regroupées dans une seule clé "Image_url"
for product_id, product_data in dict_results.items():
    # Regrouper les URLs d'images par couleur sous une même clé 'Image_url'
    image_urls = {}
    for color in product_data['available colors']:
        image_key = f"Image_url_{color}"
        if image_key in product_data:
            image_urls[color] = product_data[image_key]
            # Supprimer les anciennes clés d'images après les avoir regroupées
            del product_data[image_key]
    
    # Ajouter la nouvelle clé "Image_url" contenant les images regroupées
    product_data['Image_url'] = image_urls

# Afficher le dictionnaire modifié
from pprint import pprint
pprint(dict_results)

# Build a DataFrame with the results

In [ ]:
# Transformation du dictionnaire en DataFrame
df = pd.DataFrame.from_dict(dict_results, orient='index')

# Affichage des données
df.head()


In [ ]:
# Trouver à la fois les <a> et <span> dans la pagination
page_numbers_elements = find_elements(driver, "xpath", "//a[@class='page-numbers' or @class='page-numbers current'] | //span[@class='page-numbers current']")

for page in page_numbers_elements:
    text = page.text
    if text:
        print(text)

In [ ]:
one_card = cards[0]
# Attendre que le lien de détail du produit dans "one_card" soit visible
link_to_details = wait5.until(EC.presence_of_element_located((
    By.XPATH, ".//h3[contains(@class, 'product-title')]/a"
)))

# Récupérer l'URL du produit
product_url = link_to_details.get_attribute("href")
print(product_url)

Article Details Page

In [ ]:
zone_sizes = find_element(driver, "xpath", "//ul[contains(@aria-label,'Talla')]")

# Sélectionne uniquement les tailles SANS la classe 'disabled'
available_sizes = find_elements(zone_sizes, "xpath", ".//li[not(contains(@class, 'disabled'))]")

# Affiche les tailles disponibles
for size in available_sizes:
    size_name = size.get_attribute("title")
    if size_name:
        print(size_name)

datas["available sizes"] = [size for size in available_sizes]


In [ ]:
card_one = cards[0]
title = find_element(card_one, "xpath", "//h3[contains(@class, 'product-title')]")
print(title.text)

# Promos

In [ ]:
promos_page_links = [
    'https://renzo.com.co/categoria-producto/ofertas-ropa-hombre-ninos/sale-renzo/',
    'https://renzo.com.co/categoria-producto/ofertas-ropa-hombre-ninos/sale-bambino/'
]

for item in promos_page_links:
    try:
        promos_hombres = find_element(choice_promos, 'xpath', 
            f".//a[contains(@href, '{item}')]")
        print("[OK] Élément trouvé pour promos_hombres")
    except Exception as e:
        print(f"[!!] Élément manquant pour promos_hombres : {e}")

    try:
        promos_ninos = find_element(choice_promos, 'xpath', 
            f".//a[contains(@href, '{item}')]")
        print("[OK] Élément trouvé pour promos_ninos")
    except Exception as e:
        print(f"[!!] Élément manquant pour promos_ninos : {e}")
